In [ ]:
import pandas as pd
import time
import os
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:
file_input = f""
file_output = f""
download_dir = "\\data\\get_mf\\fatos_relevantes" 

In [ ]:
df = pd.read_csv(file_input)


print(len(df))
print()
df.head()

In [ ]:
options = Options()

# Set preferences for download
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "*/*")  
options.set_preference("browser.helperApps.alwaysAsk.force", False)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.folderList", 2)  # Use a custom download directory
options.set_preference("browser.download.dir", download_dir)  # Path to save the files
options.set_preference("pdfjs.disabled", True)  # Disable Firefox's built-in PDF viewer

options.set_preference("browser.download.useDownloadDir", True)
options.set_preference("browser.download.panel.shown", False)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.manager.focusWhenStarting", False)
options.set_preference("browser.download.manager.showAlertOnComplete", False)
options.set_preference("browser.download.manager.closeWhenDone", True)



In [ ]:
def wait_for_downloads(download_path):
    while any([filename.endswith(".part") for filename in os.listdir(download_path)]):
        time.sleep(1)

# Iterar sobre os links no dataframe
for index, row in df.iterrows():
    try:
        
        # Initialize Firefox driver with options
        driver = webdriver.Firefox(options=options)
        # Abrir o URL
        url = row['fr_link']
        driver.get(url)

        # Dar um tempo 
        time.sleep(3)

        # Esperar o arquivo ser totalmente baixado
        wait_for_downloads(download_dir)

        # Renomear o arquivo baixado
        saved_filename = None
        for filename in os.listdir(download_dir):
            if filename.endswith(".pdf") and "documento_" not in filename:
                old_path = os.path.join(download_dir, filename)
                new_filename = f"documento_{index}.pdf"  # Nome do arquivo com base no índice
                new_path = os.path.join(download_dir, new_filename)
                os.rename(old_path, new_path)
                saved_filename = new_filename
                break

        df.at[index, 'fr_pdf'] = saved_filename
        
        driver.quit()
        
        if index % 50 == 0:
            print(f'''Progress {index}''')

    except Exception as e:
        print(f'Erro na linha {index}: {e}')
        df.at[index, 'fr_pdf'] = "nan"
        
        driver.quit()
        
        continue

# Fechar o navegador
driver.quit()




In [ ]:
df.head()

In [ ]:
df.to_csv(file_output, index=False)